## Installation

In [193]:
#pip install utilities-package

In [194]:
#pip install gensim nltk

In [196]:
import gensim
import re
import numpy as np
from nltk import ngrams
from utilities import *

In [198]:
#pip install fasttext

In [197]:
# ============================   
# ====== N-Grams Models ======

#t_model = gensim.models.Word2Vec.load('C:/Users/User/OneDrive/Bureau/full_grams_cbow_100_twitter/full_grams_cbow_100_twitter.mdl')

In [105]:
from gensim.models import KeyedVectors
import fasttext.util

In [199]:
#fasttext.util.download_model('ar', if_exists='ignore')

In [200]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [174]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Charger les modèles de plongement lexical pré-entraînés
unigram_model = gensim.models.KeyedVectors.load_word2vec_format("fast_text_model.vec", binary=False)
bigram_model = gensim.models.KeyedVectors.load_word2vec_format("model.vec", binary=False)

In [182]:
import gensim
file_path = 'w2v_SG_300_5_400_10.model'
word_embed_1 = gensim.models.Word2Vec.load(file_path) #load_model

In [183]:
file_path = 'w2v_CBOW_300_5_400_10.model'
word_embed_2 = gensim.models.Word2Vec.load(file_path)

In [186]:
# Save the model as a vector file
word_embed_1.wv.save_word2vec_format('model_SG.vec')

In [187]:
# Save the model as a vector file
word_embed_2.wv.save_word2vec_format('model_CBOW.vec')

In [3]:
import res
from gensim.models import KeyedVectors

model_fasttext = KeyedVectors.load_word2vec_format("fast_text_model.vec")
model_ = KeyedVectors.load_word2vec_format("model.vec")
model_CBOW = KeyedVectors.load_word2vec_format("model_CBOW.vec")
model_SG = KeyedVectors.load_word2vec_format("model_SG.vec")

In [8]:
import re
from nltk.metrics.distance import edit_distance
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer


## Models

### One Model 

In [84]:
ft_model = model_  

In [85]:
input_sentence = "يحإب علينا الذهاب باكرا "   

In [118]:
## Prétraitement 

input_sentence = re.sub(r'[^\w\s]','',input_sentence) # Supprimer les caractères spéciaux
input_sentence = input_sentence.lower() # Convertir en minuscule

# Initialisation des outils de traitement du langage naturel
stop_words = set(stopwords.words('arabic'))
stemmer = SnowballStemmer("arabic")

# Initialisation des listes pour stocker les résultats
corrected_sentence = []
potential_errors = []
correction = []
d = {}

In [119]:
vector_word = []
for word in input_sentence.split():
    if word in stop_words: # ignorer les mots vides
        continue
    try:
        vector = ft_model.get_vector(word)  # création du vecteur de plangemnt lexical de chaque mot 
    except KeyError:
        vector = np.zeros(ft_model.vector_size)  # si le mots ne fait pas partie de l'ensemble des mots connues par le modèle on remplit le vecteur avec des 0
    vector_word.append(vector)
    
## afficher le mot erroné
for i in range(len(vector_word)) :
    if vector_word[i].all() == np.zeros(ft_model.vector_size).all() : 
        err_word = input_sentence.split()[i]
        print( err_word, "n'est pas connus")
        
        
## prédir la correction en se basant sur le vocabulaire du model et la distance minimal entre le lemme du mot et les autres mots du vocabulaire 
stemmed_word = stemmer.stem(err_word) # racine du mot erroné
min_distance = float('inf')   # initialiser la valeur minimal de distance 
closest_word = None

for vocab_word in ft_model.index_to_key:  ## parcourir le vocabulaire du modèle mis en évidence 
    stemmed_vocab_word = stemmer.stem(vocab_word)
    distance = edit_distance(stemmed_word, stemmed_vocab_word)
    if distance < min_distance:
        min_distance = distance
        closest_word = vocab_word
        correction.append(closest_word)
        d[min_distance] = vocab_word


يحإب n'est pas connus


In [138]:
## Afficher les deux mots les plus proches prédit comme correction
for i in list(d.keys())[len(d.keys())-2:len(d.keys())]:
    print(i,d[i])

1 يحب
0 يحابي


### Two Models

In [165]:
model1 = model_fasttext
model2 = model_

In [166]:
input_sentence = "يحإب علينا الذهاب باكرا "   

In [167]:
## Prétraitement 

input_sentence = re.sub(r'[^\w\s]','',input_sentence) # Supprimer les caractères spéciaux
input_sentence = input_sentence.lower() # Convertir en minuscule

# Initialisation des outils de traitement du langage naturel
stop_words = set(stopwords.words('arabic'))
stemmer = SnowballStemmer("arabic")

# Initialisation des listes pour stocker les résultats
corrected_sentence = []
potential_errors = []
correction = []
d = {}

In [168]:
vector_word = []
## création du vécteur de plangement lexicale de chaque mot en faisant la concaténation entre les vecteurs obtenue de chaque modèle
for word in input_sentence.split():
    if word in stop_words: # ignorer les mots vides
        continue
    try:
        vector1 = model1.get_vector(word)  # création du vecteur de plangemnt lexical de chaque mot 
    except KeyError:
        vector1 = np.zeros(model1.vector_size)  # si le mots ne fait pas partie de l'ensemble des mots connues par le modèle on remplit le vecteur avec des 0
    try:
        vector2 = model2.get_vector(word)
    except KeyError:
        vector2 = np.zeros(model2.vector_size)
    combined_vector = np.concatenate([vector1, vector2]) ## la dimention = dim(v1) + dim(v2) (=200 dans notre cas)
    vector_word.append(combined_vector)
    

In [170]:
## afficher le mot erroné
for i in range(len(vector_word)) :
    if vector_word[i].all() == np.zeros(len(vector_word[0])).all() : 
        err_word = input_sentence.split()[i]
        print( err_word, "n'est pas connus")

يحإب n'est pas connus


In [150]:
## prédir la correction en se basant sur le vocabulaire du model et la distance minimal entre le lemme du mot et les autres mots du vocabulaire 
stemmed_word = stemmer.stem(err_word) # racine du mot erroné
min_distance = float('inf')   # initialiser la valeur minimal de distance 
closest_word = None

In [151]:
for model in [model1,model2]:
    for vocab_word in model.index_to_key:  ## parcourir le vocabulaire du modèle mis en évidence 
        stemmed_vocab_word = stemmer.stem(vocab_word)
        distance = edit_distance(stemmed_word, stemmed_vocab_word)
        if distance < min_distance:
            min_distance = distance
            closest_word = vocab_word
            correction.append(closest_word)
            d[min_distance] = vocab_word

In [153]:
## Afficher les deux mots les plus proches prédit comme correction
for i in list(d.keys())[len(d.keys())-2:len(d.keys())]:
    print(i,d[i])

1 يحب
0 يحابي


### Three Models

In [177]:
model1 = model_fasttext
model2 = model_
model3 = model_CBOW

In [178]:
input_sentence = "يحإب علينا الذهاب باكرا "   

In [179]:
## Prétraitement 

input_sentence = re.sub(r'[^\w\s]','',input_sentence) # Supprimer les caractères spéciaux
input_sentence = input_sentence.lower() # Convertir en minuscule

# Initialisation des outils de traitement du langage naturel
stop_words = set(stopwords.words('arabic'))
stemmer = SnowballStemmer("arabic")

# Initialisation des listes pour stocker les résultats
corrected_sentence = []
potential_errors = []
correction = []
d = {}

In [180]:
vector_word = []
## création du vécteur de plangement lexicale de chaque mot en faisant la concaténation entre les vecteurs obtenue de chaque modèle
for word in input_sentence.split():
    if word in stop_words: # ignorer les mots vides
        continue
    try:
        vector1 = model1.get_vector(word)  # création du vecteur de plangemnt lexical de chaque mot 
    except KeyError:
        vector1 = np.zeros(model1.vector_size)  # si le mots ne fait pas partie de l'ensemble des mots connues par le modèle on remplit le vecteur avec des 0
    try:
        vector2 = model2.get_vector(word)
    except KeyError:
        vector2 = np.zeros(model2.vector_size)
    try:
        vector3 = model3.get_vector(word)
    except KeyError:
        vector3 = np.zeros(model3.vector_size)
    combined_vector = np.concatenate([vector1, vector2,vector3]) ## la dimention = dim(v1) + dim(v2)+dim(v3)
    vector_word.append(combined_vector)

In [181]:
## afficher le mot erroné
for i in range(len(vector_word)) :
    if vector_word[i].all() == np.zeros(len(vector_word[0])).all() : 
        err_word = input_sentence.split()[i]
        print( err_word, "n'est pas connus")

يحإب n'est pas connus


In [182]:
## prédir la correction en se basant sur le vocabulaire du model et la distance minimal entre le lemme du mot et les autres mots du vocabulaire 
stemmed_word = stemmer.stem(err_word) # racine du mot erroné
min_distance = float('inf')   # initialiser la valeur minimal de distance 
closest_word = None

In [183]:
for model in [model1,model2,model3]:
    for vocab_word in model.index_to_key:  ## parcourir le vocabulaire du modèle mis en évidence 
        stemmed_vocab_word = stemmer.stem(vocab_word)
        distance = edit_distance(stemmed_word, stemmed_vocab_word)
        if distance < min_distance:
            min_distance = distance
            closest_word = vocab_word
            correction.append(closest_word)
            d[min_distance] = vocab_word

In [184]:
## Afficher les deux mots les plus proches prédit comme correction
for i in list(d.keys())[len(d.keys())-2:len(d.keys())]:
    print(i,d[i])

1 يحب
0 يحابي


### Four models

In [185]:
model1 = model_fasttext
model2 = model_
model3 = model_CBOW
model4 = model_SG

In [186]:
input_sentence = "يحإب علينا الذهاب باكرا "   

In [187]:
## Prétraitement 

input_sentence = re.sub(r'[^\w\s]','',input_sentence) # Supprimer les caractères spéciaux
input_sentence = input_sentence.lower() # Convertir en minuscule

# Initialisation des outils de traitement du langage naturel
stop_words = set(stopwords.words('arabic'))
stemmer = SnowballStemmer("arabic")

# Initialisation des listes pour stocker les résultats
corrected_sentence = []
potential_errors = []
correction = []
d = {}

In [188]:
vector_word = []
## création du vécteur de plangement lexicale de chaque mot en faisant la concaténation entre les vecteurs obtenue de chaque modèle
for word in input_sentence.split():
    if word in stop_words: # ignorer les mots vides
        continue
    try:
        vector1 = model1.get_vector(word)  # création du vecteur de plangemnt lexical de chaque mot 
    except KeyError:
        vector1 = np.zeros(model1.vector_size)  # si le mots ne fait pas partie de l'ensemble des mots connues par le modèle on remplit le vecteur avec des 0
    try:
        vector2 = model2.get_vector(word)
    except KeyError:
        vector2 = np.zeros(model2.vector_size)
    try:
        vector3 = model3.get_vector(word)
    except KeyError:
        vector3 = np.zeros(model3.vector_size)
    try:
        vector4 = model4.get_vector(word)
    except KeyError:
        vector4 = np.zeros(model4.vector_size)
    combined_vector = np.concatenate([vector1, vector2,vector3,vector4]) ## la dimention = dim(v1) + dim(v2) (=200 dans notre cas)
    vector_word.append(combined_vector)

In [189]:
## afficher le mot erroné
for i in range(len(vector_word)) :
    if vector_word[i].all() == np.zeros(len(vector_word[0])).all() : 
        err_word = input_sentence.split()[i]
        print( err_word, "n'est pas connus")

يحإب n'est pas connus


In [190]:
## prédir la correction en se basant sur le vocabulaire du model et la distance minimal entre le lemme du mot et les autres mots du vocabulaire 
stemmed_word = stemmer.stem(err_word) # racine du mot erroné
min_distance = float('inf')   # initialiser la valeur minimal de distance 
closest_word = None

In [191]:
for model in [model1,model2,model3,model4]:
    for vocab_word in model.index_to_key:  ## parcourir le vocabulaire du modèle mis en évidence 
        stemmed_vocab_word = stemmer.stem(vocab_word)
        distance = edit_distance(stemmed_word, stemmed_vocab_word)
        if distance < min_distance:
            min_distance = distance
            closest_word = vocab_word
            correction.append(closest_word)
            d[min_distance] = vocab_word

In [192]:
## Afficher les deux mots les plus proches prédit comme correction
for i in list(d.keys())[len(d.keys())-2:len(d.keys())]:
    print(i,d[i])

1 يحب
0 يحابي
